In [1]:
import json

# Booth

In [43]:
class Booth:
  def __init__(self, id:str, name:str, num_round:int=4, closed_rounds:list=None):
    self.__id = id
    self.__name = name
    self.__num_round = num_round
    self.__closed_rounds = list()
    self.__rounds = dict()
    for i in range(self.__num_round):
      self.__rounds[i] = 0

  def get_info(self):
    return { 
      'id':self.__id, 
      'name':self.__name,
      'num_round':self.__num_round,
      'closed_rounds':self.__closed_rounds,
    }

  def close_rounds(self, rounds:list):
    for round in rounds:
      if (0 < round <= self.__num_round):
        self.__closed_rounds.append(round - 1)
      else:
        raise Exception('One of closed rounds is out of range.')
    print(f"Closed round {', '.join([str(round) for round in rounds])}")

  def get_rounds(self):
    return self.__rounds

  def add_student_to_round(self, round:int):
    self.__rounds[round] += 1 

  def cal_round_weights(self, ordered:bool=False):
    round_weights = list()
    for round, weight in self.__rounds.items():
      if (round in self.__closed_rounds):
        round_weight = {'round':round, 'weight':weight + 1000}
      else:
        round_weight = {'round':round, 'weight':weight}
      round_weights.append(round_weight)
    if (ordered):
      round_weights = sorted(round_weights, key=lambda round: round['weight'])
    return round_weights
  
  def cal_booth_weight(self, round_weights:list=None):
    round_weights = round_weights if round_weights is not None \
                    else self.cal_round_weights()
    round_weights = [round_weight['weight'] for round_weight
                       in round_weights]
    return max(round_weights) - min(round_weights)


In [28]:
with open('booths.json', mode='r', encoding='utf-8') as file:
  booths_data = json.load(file)

In [44]:
Booths = dict()

for booth_group in booths_data:
  for booth in booth_group['booths']:
    print(f"Generating booth {booth['id']}...")
    Booths[booth['id']] = Booth(id=booth['id'], name=booth['name'])

Generating booth G0101...
Generating booth G0102...
Generating booth G0103...
Generating booth G0104...
Generating booth G0105...
Generating booth G0106...
Generating booth G0107...
Generating booth G0108...
Generating booth G0109...
Generating booth G0110...
Generating booth G0111...
Generating booth G0201...
Generating booth G0202...
Generating booth G0203...
Generating booth G0204...
Generating booth G0205...
Generating booth G0301...
Generating booth G0302...
Generating booth G0303...
Generating booth G0401...
Generating booth G0402...
Generating booth G0403...
Generating booth G0404...
Generating booth G0405...
Generating booth G0406...
Generating booth G0501...
Generating booth G0502...
Generating booth G0503...
Generating booth G0504...
Generating booth G0601...
Generating booth G0602...


## Close some booth rounds

In [5]:
with open('registrations.json', mode='r', encoding='utf-8') as file:
  students = json.load(file)

In [19]:
def filter_students(students:list, booths:set, intersection:int=1):
  filtered_students = list()
  for student in students:
    intersection_booths = set(booths).intersection(set(student['selectedBooths']))
    if (len(intersection_booths) >= intersection):
      filtered_students.append(student)
  return filtered_students

In [22]:
fixed_round_booths = [
  "G0104", # พยาบาลศาสตร์
  "G0109", # ทัศนมาตรศาสตร์
  "G0204", # นวัตกรรมการเกษตร
  "G0404", # นวัตกรรมศาสตร์
  "G0602", # นานาชาติจีน
]

In [23]:
fixed_round_students = filter_students(students, 
                                       booths=fixed_round_booths,
                                       intersection=2)

In [24]:
for student in fixed_round_students:
  intersection_booths = set(fixed_round_booths) \
    .intersection(set(student['selectedBooths']))
  print(intersection_booths)


{'G0602', 'G0404'}
{'G0109', 'G0104'}
{'G0109', 'G0204'}
{'G0109', 'G0104'}


  "G0104", # พยาบาลศาสตร์ = Open only round 2

  "G0109", # ทัศนมาตรศาสตร์ = Open only round 1

  "G0204", # นวัตกรรมการเกษตร = Open only round 2

  "G0404", # นวัตกรรมศาสตร์ = Open only round 1
  
  "G0602", # นานาชาติจีน = Open only round 2

In [45]:
Booths["G0104"].close_rounds([1,3,4])
Booths["G0109"].close_rounds([2,3,4])
Booths["G0204"].close_rounds([1,3,4])
Booths["G0404"].close_rounds([2,3,4])
Booths["G0602"].close_rounds([1,3,4])

Closed round 1, 3, 4
Closed round 2, 3, 4
Closed round 1, 3, 4
Closed round 2, 3, 4
Closed round 1, 3, 4


In [46]:
Booths["G0602"].get_info()

{'id': 'G0602',
 'name': 'นานาชาติจีน',
 'num_round': 4,
 'closed_rounds': [0, 2, 3]}

# Student Allocation

In [47]:
def allocate_student(student:dict):
  selected_booths = [{'id':booth} for booth in student['selectedBooths']]
  for i in range(len(selected_booths)):
    booth_id = selected_booths[i]['id']
    round_weights = Booths[booth_id].cal_round_weights(ordered=True)
    booth_weight = Booths[booth_id].cal_booth_weight(round_weights)
    selected_booths[i]['booth_weight'] = booth_weight
    selected_booths[i]['round_weights'] = round_weights
  selected_booths = sorted(selected_booths,
                           key=lambda booth:booth['booth_weight'], 
                           reverse=True)
  allocation = [None for _ in range(4)]
  for booth in selected_booths:
    booth_id = booth['id']
    for round in booth['round_weights']:
      if (allocation[round['round']] is None):
        allocation[round['round']] = booth_id
        Booths[booth_id].add_student_to_round(round['round'])
        break
  return allocation

In [48]:
allocated_students = list()

for student in students:
  allocated_booths = allocate_student(student)
  allocated_student = dict()
  allocated_student['id'] =  student['id']
  allocated_student['title'] =  student['title']
  allocated_student['firstName'] =  student['firstName']
  allocated_student['lastName'] =  student['lastName']
  allocated_student['classroom'] =  student['classroom']
  allocated_student['roll'] =  student['roll']
  allocated_student['allocated_booths'] =  allocated_booths
  allocated_students.append(allocated_student)

In [49]:
for booth in Booths:
  print(Booths[booth].get_info()['name'])
  print(Booths[booth].cal_round_weights())

แพทยศาสตร์
[{'round': 0, 'weight': 17}, {'round': 1, 'weight': 18}, {'round': 2, 'weight': 20}, {'round': 3, 'weight': 19}]
ทันตแพทยศาสตร์
[{'round': 0, 'weight': 11}, {'round': 1, 'weight': 11}, {'round': 2, 'weight': 13}, {'round': 3, 'weight': 11}]
เภสัชศาสตร์
[{'round': 0, 'weight': 13}, {'round': 1, 'weight': 11}, {'round': 2, 'weight': 14}, {'round': 3, 'weight': 14}]
พยาบาลศาสตร์
[{'round': 0, 'weight': 1000}, {'round': 1, 'weight': 17}, {'round': 2, 'weight': 1000}, {'round': 3, 'weight': 1000}]
รังสีเทคนิค
[{'round': 0, 'weight': 8}, {'round': 1, 'weight': 7}, {'round': 2, 'weight': 10}, {'round': 3, 'weight': 8}]
เทคนิคการแพทย์
[{'round': 0, 'weight': 18}, {'round': 1, 'weight': 17}, {'round': 2, 'weight': 20}, {'round': 3, 'weight': 19}]
กายภาพบำบัดและเวชศาสตร์การกีฬา
[{'round': 0, 'weight': 11}, {'round': 1, 'weight': 10}, {'round': 2, 'weight': 11}, {'round': 3, 'weight': 13}]
การแพทย์แผนตะวันออก
[{'round': 0, 'weight': 0}, {'round': 1, 'weight': 0}, {'round': 2, 'weight':

In [50]:
import json

with open('allocated_students.json', mode='w', encoding='utf-8') as file:
  json.dump(allocated_students, file, indent=2)